# Map of Working Incidents by Station Area

In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout
import numpy as np
import matplotlib.pyplot as plt
from folium import plugins
import folium

In [2]:
#import boto3

In [3]:
#s3 = boto3.resource('s3')

In [4]:
import s3fs

df = pd.read_excel('s3://pgfd-testbucket-001/WorkingIncidents2020_LatLong.xlsx')

In [5]:
#df = pd.read_excel('/Users/holliknight/Desktop/PGFD/FW Data/WorkingIncidents2020_LatLong.xlsx')

In [6]:
df.head()

,TriggerID,Time_Query,Time_CallEnteredQueue,Priority_Number,Problem,Response_Number,Radio_Name,Longitude,Latitude,AddressLocation,...,DataLinkCount_1,DataLinkCount_2,DataLinkCount_3,DataLinkCount_4,DataLinkCount_5,DataLinkCount_6,DataLinkCount_7,DataLinkCount_8,DataLinkCount_9,DataLinkCount_10
0,12474,2020-12-31 23:59:59,2020-01-03 02:42:01,0,APTT,NaN,2ALARM,76846897,39094040,9104 SCOTT ADAM CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12474,2020-12-31 23:59:59,2020-01-03 02:42:01,0,APTT,NaN,ETASK,76846897,39094040,9104 SCOTT ADAM CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12474,2020-12-31 23:59:59,2020-01-03 02:42:01,0,APTT,NaN,WFD,76846897,39094040,9104 SCOTT ADAM CT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12474,2020-12-31 23:59:59,2020-01-05 05:49:33,1,HOUSEF,NaN,WFD,76949237,38940740,4203 COTTAGE TER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12474,2020-12-31 23:59:59,2020-01-07 00:47:29,0,HOUSEF,NaN,WFD,76974982,38825261,4705 WHEELER RD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df = df[['Time_CallEnteredQueue', 'Master_Incident_Number', 'Problem', 'Radio_Name', 'Longitude', 'Latitude', 
       'Extra_1', 'Extra_10','Extra_3']]

In [12]:
df.columns = ['DateTime', 'IncNum', 'Problem', 'IncidentType', 'lng', 'lat', 'Description', 'Shift', 'BoxNum']

In [13]:
df['BoxNum'] = df['BoxNum'].astype(str)
df['BoxNum'] = df['BoxNum'].apply(lambda x: x.zfill(4))

df['Station'] = df['BoxNum'].astype(str).str[0:2]

In [14]:
df['lng'] = -df['lng'] * 0.000001

In [15]:
df['lat'] = df['lat'] * 0.000001

In [16]:
unique_station = df.Station.unique()
unique_station = np.sort(unique_station)

Station = widgets.SelectMultiple(
    options = unique_station.tolist(),
    #value = ['BAYVIEW', 'NORTHERN'],
    description='Station',
    disabled=False,
    layout = Layout(width='20%', height='80px', display='flex')
)

In [17]:
unique_type = df.IncidentType.unique()
unique_type = np.sort(unique_type)

IncType = widgets.SelectMultiple(
    options = unique_type.tolist(),
    #value = ['VANDALISM', 'ASSAULT', 'ROBBERY'],
    #rows=10,
    description='IncidentType',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)

In [18]:
def update_map(Station 
               #, IncType
              ):
    
    df1 = df.copy()
    
    latitude = 38.7965
    longitude = -76.8836

    
    df_Station = df1.loc[df['Station'].isin(np.array(Station))]
    #df_IncType = df_Station.loc[df_Station['IncidentType'].isin(np.array(IncType))]
    
    Station_unique = df_Station['Station'].value_counts()
    Station_unique = Station_unique.reset_index()
    
    #IncType_unique = df_IncType['IncidentType'].value_counts()
    #IncType_unique = IncType_unique.reset_index()


    # create map and display it
    PG_map = folium.Map(location=[latitude, longitude], zoom_start=10)

    
    incidents = plugins.MarkerCluster().add_to(PG_map)

# loop through the dataframe and add each data point to the mark cluster
    for lat, lng, label, in zip(df_Station.lat, df_Station.lng, df_Station.Station):
        folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
        ).add_to(incidents)
# show map
    display(PG_map)
    
    plt.show()

In [19]:
widgets.interactive(update_map, Station = Station, #IncType = IncType
                   )

interactive(children=(SelectMultiple(description='Station', layout=Layout(display='flex', height='80px', width…